In [556]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Random Forest
from sklearn.ensemble import RandomForestRegressor
# RMSE  (MEN_SQUARED_ERROR IS DEPRECATED)
from sklearn.metrics import root_mean_squared_error



In [557]:
# Load the pkl files
path = '../../data/processed/'
ct = pd.read_pickle(path + 'centrifuga_total.pkl')
cf = pd.read_pickle(path + 'cf.pkl')
ino = pd.read_pickle(path + 'inoculo.pkl')
pino = pd.read_pickle(path + 'preinoculo.pkl')
h = pd.read_pickle(path + 'horas.pkl')
mc = pd.read_pickle(path + 'mov_componentes.pkl')
th = pd.read_pickle(path + 'th.pkl')
of = pd.read_pickle(path + 'of.pkl')
bt = pd.read_pickle(path + 'biorreactor_total.pkl')

In [558]:
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio                  152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

# Baseline

In [559]:
# What do we drop with dropna
cf_cp = cf.copy()
print(cf_cp.shape)
cf_cp.dropna(inplace=True)
print(cf_cp.shape)
cf_cp.info()

(152, 17)
(138, 17)
<class 'pandas.core.frame.DataFrame'>
Index: 138 entries, 4 to 151
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    138 non-null    object             
 1   orden_encadenado_cf     138 non-null    int64              
 2   lote_parental_cf        138 non-null    object             
 3   id_bio                  138 non-null    object             
 4   f_h_inicio_cf           138 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              138 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         138 non-null    float64            
 7   turb_inicio_cultivo_cf  138 non-null    float64            
 8   turb_fin_cultivo_cf     138 non-null    float64            
 9   viab_final_cultivo_cf   138 non-null    float64            
 10  id_centr                138 non-null    object             
 11  centr_1_turb_cf         138 no

In [560]:
# Fit a classifier with cf data
# Split the data using train_test_splitç
# Cf cpy
cf_cp = cf.copy()
# Drop NA
#cf_cp.dropna(inplace=True)
# Drop the target
target = cf_cp['producto_1_cf'].copy()
# Drop the target
cf_cp.drop('producto_1_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
cf_cp.drop('producto_2_cf', axis=1, inplace=True)
# Drop datetimes
cf_cp.drop(['f_h_inicio_cf', 'f_h_fin_cf'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
cf_cp.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr'], axis=1, inplace=True)
#cf_cp.drop(['lote', 'id_centr'], axis=1, inplace=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(cf_cp, target, test_size=0.2, random_state=42)



In [561]:
# Fit the model
# Random Forest
def fit_predict_rf(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=300, random_state=20)
    rf.fit(X_train, y_train)
    # Predict
    y_pred = rf.predict(X_test)
    # RMSE
    rmse = root_mean_squared_error(y_test, y_pred)
    print(f'RMSE: {rmse}')
fit_predict_rf(X_train, X_test, y_train, y_test)

RMSE: 211.22088445376417


In [562]:
# Perform cross validation, with 
from sklearn.model_selection import cross_val_score
def cross_val_rf(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=100, random_state=20)
    scores = cross_val_score(rf, X_train, y_train, cv=15, scoring='neg_root_mean_squared_error')
    print(f'Cross val scores: {scores}')
cross_val_rf(cf_cp, target)

Cross val scores: [-242.23337328 -179.76189859 -199.53043999 -217.6108491  -342.34952746
 -274.85749766 -292.51523283 -320.1172982  -334.13935943 -410.84866335
 -406.78687237 -234.77269544 -248.40865932 -373.49785761 -216.65181894]


In [563]:
from lazypredict.supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np



#X, y = shuffle(boston.data, boston.target, random_state=13)
#X = X.astype(np.float32)

#offset = int(X.shape[0] * 0.9)

#X_train, y_train = X[:offset], y[:offset]
#X_test, y_test = X[offset:], y[offset:]

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:00<00:00, 103.46it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000017 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 8
[LightGBM] [Info] Start training from score 1672.990337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
RandomForestRegressor,0.05,0.34,210.67,0.05
LassoLarsIC,0.03,0.32,212.91,0.00
ElasticNet,0.03,0.32,213.67,0.00
ElasticNetCV,0.03,0.32,213.76,0.02
BayesianRidge,0.02,0.32,213.81,0.00
LassoLarsCV,0.02,0.31,214.17,0.01
LarsCV,0.02,0.31,214.17,0.01
LassoCV,0.02,0.31,214.33,0.02
LinearRegression,0.02,0.31,214.79,0.00


In [564]:
# Extract the most important features
def feat_importance(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=300, random_state=20)
    rf.fit(X_train, y_train)
    # Extract the feature importances
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = X_train.columns,
                                       columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)
feat_importance(cf_cp, target)

                        importance
turbidez_diff_cf              0.22
turb_fin_cultivo_cf           0.16
centr_2_turb_cf               0.14
centr_1_turb_cf               0.12
viab_final_cultivo_cf         0.10
dur_cf                        0.10
vol_ino_util_cf               0.09
turb_inicio_cultivo_cf        0.07
orden_encadenado_cf           0.01


In [565]:
# Can we predict the product 2?
# Fit a classifier with cf data
# Split the data using train_test_splitç
# Cf cpy
cf_cp = cf.copy()
# Drop NA
#cf_cp.dropna(inplace=True)
# Drop the target
target = cf_cp['producto_2_cf'].copy()
# Drop the target
cf_cp.drop('producto_2_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
cf_cp.drop('producto_1_cf', axis=1, inplace=True)
# Drop datetimes
cf_cp.drop(['f_h_inicio_cf', 'f_h_fin_cf'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
cf_cp.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr'], axis=1, inplace=True)
#cf_cp.drop(['lote', 'id_centr'], axis=1, inplace=True)
# dur_cf from timedelta to int

# Split the data
X_train, X_test, y_train, y_test = train_test_split(cf_cp, target, test_size=0.2, random_state=42)
# Fit the model
fit_predict_rf(X_train, X_test, y_train, y_test)
cross_val_rf(cf_cp, target)


RMSE: 1.0413941117471575
Cross val scores: [-2.36141555 -0.79761546 -1.25516046 -0.86751834 -1.69908982 -0.86982418
 -0.87015684 -1.13193536 -1.38329973 -1.24595968 -1.17917768 -1.32266258
 -1.21073274 -1.08662891 -0.92426831]


In [566]:
# Print std and mean of the target
print(f'Target std: {target.std()}')

Target std: 1.4079731610848654


In [567]:
# lazy predict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models


100%|██████████| 42/42 [00:00<00:00, 103.78it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000016 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 8
[LightGBM] [Info] Start training from score 6.194288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
NuSVR,0.25,0.47,0.94,0.00
ExtraTreesRegressor,0.20,0.44,0.97,0.03
SVR,0.15,0.40,1.00,0.00
GradientBoostingRegressor,0.12,0.38,1.01,0.03
RandomForestRegressor,0.11,0.38,1.02,0.05
LGBMRegressor,0.09,0.37,1.03,0.01
AdaBoostRegressor,0.07,0.35,1.04,0.03
HistGradientBoostingRegressor,0.07,0.35,1.04,0.02
KNeighborsRegressor,0.07,0.35,1.04,0.00


# Adding Indexed by Lote

In [568]:
# Shape of the data
print(cf_cp.shape)

(152, 9)


In [569]:
ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   lote                  167 non-null    object             
 1   id_bio                167 non-null    object             
 2   f_h_inicio_ino        163 non-null    datetime64[ns, UTC]
 3   f_h_fin_ino           163 non-null    datetime64[ns, UTC]
 4   vol_cultivo_ino       163 non-null    float64            
 5   turb_inicio_cultivo   161 non-null    float64            
 6   turb_fin_cultivo      167 non-null    float64            
 7   viab_fin_cultivo_ino  167 non-null    float64            
 8   turb_diff_ino         161 non-null    float64            
 9   dur_ino               163 non-null    float64            
dtypes: datetime64[ns, UTC](2), float64(6), object(2)
memory usage: 13.2+ KB


In [570]:
# Merge ino with cf by lote
ino_cp = ino.copy()
cf_cp = cf.copy()
# Print intersection of columns
print(set(ino_cp.columns).intersection(set(cf_cp.columns)))

{'id_bio', 'lote'}


In [571]:
# Are id_bio for ino and cf the same?
# Print unique values of id_bio
print(ino_cp['id_bio'].unique())
print(cf_cp['id_bio'].unique())
# Rename their id_bio to id_bio_ino and id_bio_cf
ino_cp.rename(columns={'id_bio': 'id_bio_ino'}, inplace=True)
cf_cp.rename(columns={'id_bio': 'id_bio_cf'}, inplace=True)


['13171' '14618' '13172']
['14615' '14616' '13170' '14614' '13169' '14617' '13189']


In [572]:

# Merge
cf_ino = cf_cp.merge(ino_cp, on=['lote'], how='left')
cf_ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    152 non-null    object             
 1   orden_encadenado_cf     152 non-null    int64              
 2   lote_parental_cf        152 non-null    object             
 3   id_bio_cf               152 non-null    object             
 4   f_h_inicio_cf           152 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              152 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         147 non-null    float64            
 7   turb_inicio_cultivo_cf  152 non-null    float64            
 8   turb_fin_cultivo_cf     152 non-null    float64            
 9   viab_final_cultivo_cf   152 non-null    float64            
 10  id_centr                152 non-null    object             
 11  centr_1_turb_cf         148 non-null    float

In [573]:
cf_ino.sample(15)

,lote,orden_encadenado_cf,lote_parental_cf,id_bio_cf,f_h_inicio_cf,f_h_fin_cf,vol_ino_util_cf,turb_inicio_cultivo_cf,turb_fin_cultivo_cf,viab_final_cultivo_cf,...,turbidez_diff_cf,id_bio_ino,f_h_inicio_ino,f_h_fin_ino,vol_cultivo_ino,turb_inicio_cultivo,turb_fin_cultivo,viab_fin_cultivo_ino,turb_diff_ino,dur_ino
135,24037,2,24031.0,13169,2024-02-19 10:20:00+00:00,2024-02-21 06:00:00+00:00,80.00,17.92,90.40,183200000.00,...,72.48,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
108,23135,2,23125.0,14616,2023-11-29 11:55:00+00:00,2023-12-01 09:00:00+00:00,80.00,19.92,77.84,164000000.00,...,57.92,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
78,23104,1,nan,14615,2023-10-31 08:15:00+00:00,2023-11-02 08:45:00+00:00,80.00,16.40,81.60,180800000.00,...,65.20,13172,2023-10-30 08:45:00+00:00,2023-10-31 08:15:00+00:00,80.00,16.16,27.76,90400000.00,11.60,84600.00
31,23051,1,nan,13170,2023-06-21 06:00:00+00:00,2023-06-23 08:30:00+00:00,80.00,14.80,49.76,211200000.00,...,34.96,14618,2023-06-20 05:40:00+00:00,2023-06-21 06:00:00+00:00,167.20,14.24,25.20,82400000.00,10.96,87600.00
101,23127,1,nan,14614,2023-11-28 06:00:00+00:00,2023-11-30 06:55:00+00:00,80.40,14.48,78.00,198400000.00,...,63.52,14618,2023-11-27 06:15:00+00:00,2023-11-28 06:00:00+00:00,80.40,14.00,28.48,134400000.00,14.48,85500.00
43,23068,1,nan,13170,2023-07-18 06:05:00+00:00,2023-07-20 05:55:00+00:00,82.00,16.40,72.96,158400000.00,...,56.56,13172,2023-07-17 05:35:00+00:00,2023-07-18 05:35:00+00:00,165.60,14.16,21.92,131200000.00,7.76,86400.00
60,23086,1,nan,14614,2023-09-25 06:15:00+00:00,2023-09-27 08:20:00+00:00,83.20,16.16,64.24,124000000.00,...,48.08,13171,2023-09-24 06:15:00+00:00,2023-09-25 06:15:00+00:00,160.00,15.44,29.44,100000000.00,14.00,86400.00
39,23063,1,nan,14615,2023-07-10 05:35:00+00:00,2023-07-12 09:35:00+00:00,83.20,17.60,68.32,113600000.00,...,50.72,14618,2023-07-09 05:45:00+00:00,2023-07-10 05:45:00+00:00,166.40,16.16,22.80,101600000.00,6.64,86400.00
149,24051,1,nan,13169,2024-03-23 07:57:00+00:00,2024-03-25 07:33:00+00:00,84.16,17.76,80.80,160800000.00,...,63.04,14618,2024-03-22 06:24:00+00:00,2024-03-23 07:28:00+00:00,168.32,14.72,25.92,102400000.00,11.20,90240.00
24,23043,1,nan,14615,2023-06-12 06:00:00+00:00,2023-06-14 08:30:00+00:00,84.00,16.24,64.96,188000000.00,...,48.72,14618,2023-06-11 05:35:00+00:00,2023-06-12 05:35:00+00:00,84.00,13.20,24.24,87200000.00,11.04,86400.00


In [574]:
# Now perform lazy predict
# Drop the target
target = cf_ino['producto_1_cf'].copy()
# Drop the target
cf_ino.drop('producto_1_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
cf_ino.drop('producto_2_cf', axis=1, inplace=True)
# Drop datetimes
cf_ino.drop(['f_h_inicio_cf', 'f_h_fin_cf', 'f_h_inicio_ino', 'f_h_fin_ino'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
cf_ino.drop(['lote', 'lote_parental_cf', 'id_bio_cf','id_bio_ino', 'id_centr'], axis=1, inplace=True)


In [575]:

# Split the data
X_train, X_test, y_train, y_test = train_test_split(cf_ino, target, test_size=0.2, random_state=42)
# Fit the model
fit_predict_rf(X_train, X_test, y_train, y_test)


RMSE: 208.12936055078637


In [576]:
# Lazy predict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:00<00:00, 90.04it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000017 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 419
[LightGBM] [Info] Number of data points in the train set: 121, number of used features: 14
[LightGBM] [Info] Start training from score 1672.990337
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
RandomForestRegressor,-0.21,0.40,200.91,0.06
LarsCV,-0.26,0.37,205.32,0.01
LassoLarsCV,-0.26,0.37,205.32,0.01
LassoCV,-0.27,0.37,205.90,0.02
TweedieRegressor,-0.27,0.37,206.21,0.00
ElasticNetCV,-0.27,0.36,206.22,0.02
BayesianRidge,-0.29,0.35,207.92,0.00
GammaRegressor,-0.29,0.35,208.06,0.01
LassoLarsIC,-0.30,0.35,208.30,0.00


In [577]:
# Perform cross validation
cross_val_rf(cf_ino, target)

Cross val scores: [-237.78676725 -163.65378141 -198.73750085 -220.93553189 -371.99440234
 -311.53703376 -274.83606072 -313.41717772 -357.51794402 -425.75998927
 -419.64786824 -245.96496623 -204.13951701 -379.53200132 -197.29145484]


In [578]:
# Feature importance
feat_importance(cf_ino, target)

                        importance
turbidez_diff_cf              0.19
turb_fin_cultivo_cf           0.13
dur_ino                       0.11
centr_2_turb_cf               0.09
centr_1_turb_cf               0.08
vol_ino_util_cf               0.06
turb_fin_cultivo              0.05
viab_final_cultivo_cf         0.05
dur_cf                        0.05
turb_inicio_cultivo_cf        0.05
vol_cultivo_ino               0.04
turb_diff_ino                 0.04
viab_fin_cultivo_ino          0.04
turb_inicio_cultivo           0.03
orden_encadenado_cf           0.01


In [579]:
# Pino
pino.info()


<class 'pandas.core.frame.DataFrame'>
Index: 284 entries, 0 to 303
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   ph_pino          284 non-null    float64            
 1   turb_pino        284 non-null    float64            
 2   f_h_inicio_pino  284 non-null    datetime64[ns, UTC]
 3   f_h_fin_pino     284 non-null    datetime64[ns, UTC]
 4   dur_pino         284 non-null    float64            
 5   lote             284 non-null    object             
dtypes: datetime64[ns, UTC](2), float64(3), object(1)
memory usage: 15.5+ KB


In [580]:
# Merge pino with cf by lote
pino_cp = pino.copy()
cf_cp = cf.copy()
# Print intersection of columns
print(set(pino_cp.columns).intersection(set(cf_cp.columns)))

{'lote'}


In [581]:
# Merge
cf_pino = cf_cp.merge(pino_cp, on=['lote'], how='left')

In [582]:
cf_pino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   lote                    256 non-null    object             
 1   orden_encadenado_cf     256 non-null    int64              
 2   lote_parental_cf        256 non-null    object             
 3   id_bio                  256 non-null    object             
 4   f_h_inicio_cf           256 non-null    datetime64[ns, UTC]
 5   f_h_fin_cf              256 non-null    datetime64[ns, UTC]
 6   vol_ino_util_cf         248 non-null    float64            
 7   turb_inicio_cultivo_cf  256 non-null    float64            
 8   turb_fin_cultivo_cf     256 non-null    float64            
 9   viab_final_cultivo_cf   256 non-null    float64            
 10  id_centr                256 non-null    object             
 11  centr_1_turb_cf         252 non-null    float

In [583]:
# Drop duplicates
cf_pino.drop_duplicates(inplace=True)

In [584]:
cf_pino.sample(15)

,lote,orden_encadenado_cf,lote_parental_cf,id_bio,f_h_inicio_cf,f_h_fin_cf,vol_ino_util_cf,turb_inicio_cultivo_cf,turb_fin_cultivo_cf,viab_final_cultivo_cf,...,centr_2_turb_cf,producto_1_cf,producto_2_cf,dur_cf,turbidez_diff_cf,ph_pino,turb_pino,f_h_inicio_pino,f_h_fin_pino,dur_pino
125,23094,1,nan,14614,2023-10-09 11:15:00+00:00,2023-10-11 11:00:00+00:00,81.20,18.08,42.80,100000000.00,...,10.08,1373.30,4.80,171900,24.72,5.41,33.76,2023-10-06 22:30:00+00:00,2023-10-08 06:00:00+00:00,113400.00
249,24049,1,nan,14617,2024-03-16 08:22:00+00:00,2024-03-18 07:23:00+00:00,83.60,18.88,72.64,164800000.00,...,17.00,1342.80,4.88,169260,53.76,5.41,27.92,2024-03-13 23:30:00+00:00,2024-03-15 06:15:00+00:00,110700.00
80,23069,1,nan,14616,2023-07-18 05:30:00+00:00,2023-07-20 06:00:00+00:00,82.00,17.04,63.60,153600000.00,...,18.40,1548.88,5.60,174600,46.56,5.58,26.48,2023-07-16 03:00:00+00:00,2023-07-17 05:35:00+00:00,95700.00
69,23063,1,nan,14615,2023-07-10 05:35:00+00:00,2023-07-12 09:35:00+00:00,83.20,17.60,68.32,113600000.00,...,15.76,1790.16,5.92,187200,50.72,5.50,30.56,2023-07-08 03:00:00+00:00,2023-07-09 05:45:00+00:00,96300.00
103,23083,1,nan,14617,2023-09-12 06:00:00+00:00,2023-09-14 06:25:00+00:00,82.80,16.96,54.16,70400000.00,...,156.96,1640.61,5.52,174300,37.20,5.42,29.20,2023-09-11 03:00:00+00:00,2023-09-12 05:45:00+00:00,96300.00
194,23137,1,nan,14614,2023-12-18 12:55:00+00:00,2023-12-20 09:15:00+00:00,81.60,16.16,56.96,280000000.00,...,20.16,526.40,4.24,159600,40.80,5.52,35.68,2023-12-15 23:30:00+00:00,2023-12-17 06:45:00+00:00,112500.00
128,23096,1,nan,14617,2023-10-17 07:00:00+00:00,2023-10-19 09:30:00+00:00,80.80,15.12,64.64,117600000.00,...,18.64,1446.71,5.84,181800,49.52,5.47,31.92,2023-10-14 22:30:00+00:00,2023-10-16 05:30:00+00:00,111600.00
52,23050,1,nan,14614,2023-06-21 06:00:00+00:00,2023-06-23 06:00:00+00:00,83.20,16.16,75.28,129600000.00,...,21.36,1755.07,6.80,172800,59.12,5.53,25.92,2023-06-19 03:00:00+00:00,2023-06-20 05:30:00+00:00,95400.00
210,24017,2,nan,14617,2024-01-22 10:00:00+00:00,2024-01-24 07:15:00+00:00,80.00,19.84,83.20,160000000.00,...,24.32,1806.96,8.00,162900,63.36,NaN,NaN,NaT,NaT,NaN
50,23049,1,nan,14616,2023-06-20 05:15:00+00:00,2023-06-22 06:45:00+00:00,82.40,14.96,68.80,163200000.00,...,24.88,1766.47,7.52,178200,53.84,5.51,29.20,2023-06-18 03:00:00+00:00,2023-06-19 05:15:00+00:00,94500.00


In [585]:
# Split the data
#X_train, X_test, y_train, y_test = train_test_split(cf_pino, target, test_size=0.2, random_state=42)
# Split by lote
# Get the unique lotes
lotes = cf_pino['lote'].unique()
target = cf_pino['producto_1_cf'].copy()
# Shuffle the lotes
np.random.shuffle(lotes)
# Split the lotes
train_lotes = lotes[:int(0.8*len(lotes))]
test_lotes = lotes[int(0.8*len(lotes)):]

# Train and test
X_train = cf_pino[cf_pino['lote'].isin(train_lotes)]
y_train = target[cf_pino['lote'].isin(train_lotes)]
X_test = cf_pino[cf_pino['lote'].isin(test_lotes)]
y_test = target[cf_pino['lote'].isin(test_lotes)]



In [589]:
# Fit the model
# Drop the target
#target = cf_pino['producto_1_cf'].copy()
# Drop the target
#cf_pino.drop('producto_1_cf', axis=1, inplace=True)
# Looks like producto_2_cf is correlated with producto_1_cf
#cf_pino.drop('producto_2_cf', axis=1, inplace=True)
# Drop datetimes
#cf_pino.drop(['f_h_inicio_cf', 'f_h_fin_cf', 'f_h_inicio_pino', 'f_h_fin_pino'], axis=1, inplace=True)
# Drop lote, orden, lote_parental, id_bio, id_centr
#cf_pino.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr'], axis=1, inplace=True)

# Drop those columns but on the train and test
drop_col = ['lote', 'lote_parental_cf', 'id_bio', 'id_centr',
              'producto_1_cf', 'producto_2_cf','f_h_inicio_cf',
              'f_h_fin_cf', 'f_h_inicio_pino', 'f_h_fin_pino']
X_train.drop(['lote', 'lote_parental_cf', 'id_bio', 'id_centr',
              'producto_1_cf', 'producto_2_cf','f_h_inicio_cf',
              'f_h_fin_cf', 'f_h_inicio_pino', 'f_h_fin_pino'], axis=1, inplace=True)
X_test.drop(['lote', 'lote_parental_cf', 'id


KeyError: 'producto_1_cf'

In [588]:
# Fit the model
fit_predict_rf(X_train, X_test, y_train, y_test)

TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [492]:
# Cross val
cross_val_rf(cf_pino, target)

Cross val scores: [-285.62777649 -118.63903458 -246.10530963 -184.62171384 -257.07412931
 -303.65813192 -321.05341778 -235.7013575  -373.28795141 -333.28140714
 -440.40915883 -392.32443211 -274.47458035 -303.83613202 -206.40844282]


In [493]:
# Lazy predict
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, random_state=69)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 42/42 [00:00<00:00, 74.71it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 486
[LightGBM] [Info] Number of data points in the train set: 204, number of used features: 12
[LightGBM] [Info] Start training from score 1638.592376
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.77,0.82,131.74,0.04
XGBRegressor,0.75,0.81,135.71,0.02
GradientBoostingRegressor,0.73,0.79,142.49,0.04
BaggingRegressor,0.69,0.76,152.24,0.01
RandomForestRegressor,0.69,0.76,152.98,0.08
HistGradientBoostingRegressor,0.67,0.75,156.49,0.02
LGBMRegressor,0.67,0.75,156.73,0.01
AdaBoostRegressor,0.51,0.62,192.22,0.04
DecisionTreeRegressor,0.43,0.56,207.19,0.00


In [494]:
# feat importance
feat_importance(cf_pino, target)

                        importance
turbidez_diff_cf              0.22
dur_pino                      0.14
turb_fin_cultivo_cf           0.13
viab_final_cultivo_cf         0.10
centr_2_turb_cf               0.09
centr_1_turb_cf               0.07
dur_cf                        0.06
vol_ino_util_cf               0.06
turb_inicio_cultivo_cf        0.06
turb_pino                     0.04
ph_pino                       0.02
orden_encadenado_cf           0.00
